## Imports

In [ ]:
import sys
import pandas as pd
import numpy as np
import pickle

from functools import partial
import datapane as dp

import sklearn.pipeline as skpip
from sklearn.compose import ColumnTransformer
from category_encoders import WOEEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Import my libs
sys.path.append('../')

from src.features.balance import BalanceMixin
from src.features.correlation import HighCorrelation_filter
from src.features.variance import NearZeroVar_filter, LowVar_Filter
from src.features.passthrough import Passthrough

from src.models.binary_classification.algos.logistic_autotuning import LogisticAutoTuning
from src.models.binary_classification.algos.xgboost_autotuning import XGBoostAutoTuning

modelisationTuple

In [ ]:
with open('../data/processed/modelling_tuple', 'rb') as f:
    M = pickle.load(f)

## Preprocessing Steps

#### Function I use in preprocessing

In [ ]:
def _intersect(df: pd.DataFrame, target: list):
    """
    Intersection between a list and a set of columns in a dataframe.

    Args:
        df (pd.DataFrame): The dataframe with which we want to make the intersection.
        target (list): The array with which we want to make the intersection.
    """

    return [c for c in df.columns if c in target]

####  Split the features to be processed by their types

In [ ]:
#M.train_X.dtypes

In [ ]:
# variables that do not require preprocessing
passthrough_cols = ['']

# non-ordinal / categorical variables
categorical_cols = [''] 

# quantitative variables
quantitative_cols = set(M.train_X.select_dtypes(include=['float','int']).columns.tolist()) - set(passthrough_cols) - set(categorical_cols)
quantitative_cols = list(quantitative_cols)

####  Fast checkpoint

In [ ]:
# Check if dataframe contains NaN value
test = M.train_X.isna().sum().to_frame('nb')
test[test['nb'] > 0]

In [ ]:
 # Check if dataframe contains infinity
test = np.isinf(M.train_X).sum().to_frame('nb')
test[test['nb'] > 0]

In [ ]:
 # prevalence
y = M.train_y
out = pd.concat([
    np.round(y.value_counts(normalize=True, ascending=False).rename('normalized'), 4) * 100,
    y.value_counts(normalize=False, ascending=False).rename('number'),
], axis=1)
out

#### Preprocessor

In [ ]:
def build_data_preprocessor(passthrough_cols: list, categorical_cols: list, quantitative_cols: list, scale=False):
    """
    Creates a data preprocessing pipeline

    Args:
        passthrough_cols (list):  list of variables that do not require preprocessing.
        categorical_cols (list):  list of non-ordinal / categorical variables .
        quantitative_cols (list): list of quantitative variables.
        scale (boolean, optional): Should we standardize the variables? Defaults to False.
    """

    # This step will make each preprocessing step robust if it does not receive all the assumed variables
    passthrough_cols_selector = partial(_intersect, target=passthrough_cols)
    categorical_cols_selector = partial(_intersect, target=categorical_cols)
    quantitative_cols_selector = partial(_intersect, target=quantitative_cols)

    # Make the preprocessor pipeline
    lowvar_transformer = skpip.Pipeline(steps=[
        ('NZV', NearZeroVar_filter(equisample=True))
    ])

    # Categorical-non-ordinal pipeline
    categorical_transformer = skpip.Pipeline(steps=[
        ('encoder', WOEEncoder())
    ])

    # Passthrough pipeline
    passthrough_transformer = skpip.Pipeline(steps=[
        ('passthrough', Passthrough())
    ])

    # Quantitative pipeline
    quantitative_transformer = skpip.Pipeline(steps=[
        ('low-var', LowVar_Filter(equisample=True)),
        ('high-correlation', HighCorrelation_filter(equisample=True)),
        ('imputer', SimpleImputer(strategy='median')),
    ])

    if scale:
        quantitative_transformer.steps.append(('scaler', StandardScaler()))


    # Map the columns to their respective transformers
    # ColumnTransformer allows to apply data transformations to different features of a same df. 
    # Columns not specified in the "Transformers" list are removed from the default dataset.
    columns_transformers = ColumnTransformer(
        transformers=[
            ('quantitative', quantitative_transformer, quantitative_cols_selector),
            ('passthrough', passthrough_transformer, passthrough_cols_selector),
            ('categorical', categorical_transformer, categorical_cols_selector),
        ],
        verbose_feature_names_out=False,
    )

    # Define the pipeline
    preprocessor = skpip.Pipeline(steps=[
        ('lowvar_transformer', lowvar_transformer),
        ('columns_transformers', columns_transformers),
    ]
    )

    return preprocessor

#### SmokeTest

In [ ]:
preprocessor = build_data_preprocessor(passthrough_cols=passthrough_cols, categorical_cols=categorical_cols, quantitative_cols=quantitative_cols, scale=False)
preprocessor.fit(M.train_X, M.train_y)

In [ ]:
print(type(preprocessor))
print(type(preprocessor.named_steps['columns_transformers']))

In [ ]:
# Extract the columns names from the FINAL layer of the transformer
preprocessor.get_feature_names_out()

In [ ]:
X_transformed = preprocessor.transform(M.train_X)
X_transformed = pd.DataFrame(X_transformed, columns=preprocessor.get_feature_names_out())

## Training

#### Logistic model

In [ ]:
logistic_pipeline = skpip.Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticAutoTuning(optuna_trials=10, optim_metric_strategy='aucpr'))      
])

logistic_pipeline.fit(M.train_X, M.train_y)

In [ ]:
logistic_pipeline.named_steps['model'].model_params

In [ ]:
# Beta values in Logistic Regression
logistic_pipeline.named_steps['model']._model.coef_

In [ ]:
logistic_pipeline.named_steps['model']._threshold

#### XGBoost

In [ ]:
xgboost_pipeline  = skpip.Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBoostAutoTuning(optuna_trials=10, optim_metric_strategy='aucpr'))      
])

xgboost_pipeline .fit(M.train_X, M.train_y)

In [ ]:
xgboost_pipeline .named_steps['model'].model_params

In [ ]:
xgboost_pipeline .named_steps['model']._threshold

## Reporting

#### Functions

In [ ]:
def compute_prevalence_rate(y: np.ndarray):
    """
    Compute the empirical prevalence rate
    """

    y = pd.Series(y)
    out = pd.concat([
        np.round(y.value_counts(normalize=True, ascending=False).rename('normalized'), 4) * 100,
        y.value_counts(normalize=False, ascending=False).rename('number'),
    ], axis=1)
    
    return dp.Table(out, caption='Empirical prevalence rate')

In [ ]:
def report_preprocessing_step(preprocessor: skpip.Pipeline, prevalence_group: dp.Table):
    """
    Build the preprocessing group
    """

    # Prepare the group of steps to be iterated over
    groups = []

    # Fetch the NZV group common to all variables
    group = dp.Group(preprocessor
            .named_steps['lowvar_transformer'] \
            .named_steps['NZV'] \
            .get_report_group()
        )
    groups.append(group)

    # Iter over each group of preprocessor specific to the quantitative pipeline
    for step in ('high-correlation', 'low-var'): 
        group = dp.Group(preprocessor
            .named_steps['columns_transformers'] \
            .named_transformers_['quantitative'] \
            .named_steps[step] \
            .get_report_group()
        )
    
        groups.append(group)
    

    # compute the final number of variables keeped by the model
    cols = len(preprocessor.named_steps['columns_transformers'].get_feature_names_out())

    return dp.Group(
        dp.Text('# Data preprocessing report'),
        dp.Text('## Estimated prevalence rate'),
        prevalence_group,
        dp.BigNumber(heading='Final number of variables used by the model after preprocessing', value=cols),
        *groups)

#### SmokeTest

In [ ]:
prevalence_group = compute_prevalence_rate(M.train_y)

In [ ]:

preprocessing_group = report_preprocessing_step(preprocessor, prevalence_group)

In [ ]:
# save as a static HTML-based report
dp.save_report(preprocessing_group, path='../reports/html/prepro.html')